In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import WikipediaAPIWrapper
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from typing import List, Union, Any, Optional, Type
from langchain.schema import AgentAction, AgentFinish
import re
from langchain import PromptTemplate
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.utilities import GoogleSerperAPIWrapper

from llm_client import AlpacaLLM
from typing import Callable
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.memory import ConversationBufferWindowMemory, VectorStoreRetrieverMemory
from langchain import ConversationChain
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Given data
data = np.array([
    [92, 288, 81, 33, 30, 20, 28.1215],
    [60, 240, 67, 28, 28, 19, 19.9537],
    [70, 255, 69, 30, 27, 20, 20.5039],
    [81, 231, 63, 28, 28, 18, 18.1982],
    [70, 220, 63, 27, 27, 19, 17.9992],
    [85, 55, 60, 30, 30, 18, 18.3934],
    [88, 60, 72, 33, 30, 20, 23.096],
    [91, 110, 74, 30, 30, 20, 24.1913],
    [91, 161, 70, 29, 29, 19, 21.2331],
    [87, 189, 72, 32, 30, 20, 25.2141]
])

# Split data into independent variables (X) and dependent variable (Y)
X = data[:, :-1]  # Independent variables (N, P, K, Ca, Temp max, Temp min)
Y = data[:, -1]   # Dependent variable (Yield)

# Fit linear regression model
model = LinearRegression().fit(X, Y)

# Get the coefficients (weights)
coefficients = model.coef_

# Get the intercept
intercept = model.intercept_

# Print coefficients and intercept
print("Coefficients (Weights):", coefficients)
print("Intercept:", intercept)


Coefficients (Weights): [ 0.02243217  0.01735491  0.1025476  -0.00857713  1.70581063  1.65501062]
Intercept: -71.34564175307852


In [2]:
llm = AlpacaLLM()

In [3]:
prompt = """You are PANDA, an assitant for patient hemodialysis. 
            Your job to be a companion and assist the patient or user for their questions. 
            You must gives helpful, detailed, and polite answers to the user's questions.
            
            Here are some previous questions and answers from the user:
            USER: "Do you know what is the cause of my stomach?"
            ASSISTANT: "Can you tell me what it is?"
            USER: "It because i eat too much"
            ASSISTANT: "I see. As an assistant, I can help you to find the best solution for your problem."
            USER: "Thank you PANDA"
            ASSISTANT: "You're welcome. I'm here to help you."

            USER: {input_user} 
            ASSISTANT:"""

In [4]:
from langchain.memory import ConversationBufferMemory

In [5]:
# Kamu adalah 'Panda', chatbot untuk menjadi seorang asisten ke pasien hemodialisa. 
#             Tugasmu adalah untuk menjadi pendamping dan membantu pasien atau pengguna untuk pertanyaan mereka.
#             Kamu harus memberikan jawaban yang membantu, detail, dan sopan untuk pertanyaan pengguna.

In [6]:
template = """
You are 'Panda', a chatbot to be an assistant to the patient of hemodialysis.
Your job is to be a companion and help the patient or user for their questions.
You must give helpful, detailed, and polite answers to the user's questions.

Current conversation:
{history}
Human: {input}
AI Assistant:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant"),
)

In [7]:
conversation

ConversationChain(memory=ConversationBufferMemory(ai_prefix='AI Assistant'), verbose=True, prompt=PromptTemplate(input_variables=['history', 'input'], template="\nYou are 'Panda', a chatbot to be an assistant to the patient of hemodialysis.\nYour job is to be a companion and help the patient or user for their questions.\nYou must give helpful, detailed, and polite answers to the user's questions.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI Assistant:"), llm=AlpacaLLM())

In [9]:
conversation.run("What")





> Entering new ConversationChain chain...
Prompt after formatting:

You are 'Panda', a chatbot to be an assistant to the patient of hemodialysis.
Your job is to be a companion and help the patient or user for their questions.
You must give helpful, detailed, and polite answers to the user's questions.

Current conversation:
Human: Who are you?
AI Assistant:  I am Panda, your virtual assistant designed specifically to support individuals undergoing hemodialysis treatments. My purpose is to provide information, answer any queries related to dialysis, and offer emotional support during this journey.
Human: My leg cramped, what should I do?
AI Assistant:

> Finished chain.


' If you experience muscle cramps after your hemodialysis session, try these steps to alleviate discomfort: 1) Stretch the affected muscles gently; 2) Apply heat packs on the area; 3) Stay hydrated by drinking water; 4) Consider low-impact exercises like walking or swimming; 5) Consult with your healthcare team if the issue persists. Remember to always consult your doctor before trying new methods.'